Importing the required libraries and loading the data from my github account

In [1]:
import pandas as pd
from dash import Dash
from dash import html, dcc
from dash import callback, Output, Input
import plotly.express as px

url = 'https://raw.githubusercontent.com/gretazsikla/ceu_business_analytics_zsikla_greta/main/Data_visualization_in_python/dataframe.xlsx'

df = pd.read_excel(url, engine='openpyxl') 

In [2]:
df

,Country Name,Country Code,Time,Time Code,"Population, total [SP.POP.TOTL]",Renewable energy consumption (% of total final energy consumption) [EG.FEC.RNEW.ZS],Carbon dioxide (CO2) emissions excluding LULUCF per capita (t CO2e/capita) [EN.GHG.CO2.PC.CE.AR5]
0,Afghanistan,AFG,1990.0,YR1990,10694796.0,23,0.270833
1,Afghanistan,AFG,1991.0,YR1991,10745167.0,23.7,0.257446
2,Afghanistan,AFG,1992.0,YR1992,12057433.0,27.4,0.139549
3,Afghanistan,AFG,1993.0,YR1993,14003760.0,28.5,0.114848
4,Afghanistan,AFG,1994.0,YR1994,15455555.0,30.1,0.099369
...,...,...,...,...,...,...,...
6720,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6721,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6722,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6723,Data from database: World Development Indicators,NaN,NaN,NaN,NaN,NaN,NaN


Names are too long and consists spaces so I rename the columns

In [3]:
df.rename(columns={"Carbon dioxide (CO2) emissions excluding LULUCF per capita (t CO2e/capita) [EN.GHG.CO2.PC.CE.AR5]": "co2_emission",
                   "Renewable energy consumption (% of total final energy consumption) [EG.FEC.RNEW.ZS]":"renewable_energy_consumption",
                   "Country Name" : "country",
                   "Population, total [SP.POP.TOTL]": "population"
                  }, inplace=True)

For the app interactivity I make a new column where I categorize the population by quantiles

In [4]:
df['population_category'] = pd.qcut(df['population'], q=5, labels=['Very Low', 'Low', 'Medium', 'High', 'Very High'])

In [5]:
df

,country,Country Code,Time,Time Code,population,renewable_energy_consumption,co2_emission,population_category
0,Afghanistan,AFG,1990.0,YR1990,10694796.0,23,0.270833,High
1,Afghanistan,AFG,1991.0,YR1991,10745167.0,23.7,0.257446,High
2,Afghanistan,AFG,1992.0,YR1992,12057433.0,27.4,0.139549,High
3,Afghanistan,AFG,1993.0,YR1993,14003760.0,28.5,0.114848,High
4,Afghanistan,AFG,1994.0,YR1994,15455555.0,30.1,0.099369,High
...,...,...,...,...,...,...,...,...
6720,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6721,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6722,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6723,Data from database: World Development Indicators,NaN,NaN,NaN,NaN,NaN,NaN,NaN


My hypothesis is that as time passes the countries use more and more renewable energy so their CO2 emission decreases

In [40]:
animated = px.scatter(df, x="renewable_energy_consumption", y="co2_emission", animation_frame="Time", size="population",
                      hover_name="country", ) #creating the base chart


pop_cat = list(df.population_category.unique()) #creating a list of the names of the possible population categories
pop_cat
 
app = Dash(__name__)
 
app.layout = html.Div([
    html.H1('Relationship of CO2 emission and renewable energy consumption(%)', style={'textAlign':'center'}), #header
    html.Label('', style={'font-weight': 'bold', "text-align": "center"}),
    ## Dropdown menu for selecting population category to filter data
    dcc.Dropdown(id= 'input1', options=pop_cat, value='Medium', style={'width':'35%'}),
    dcc.Graph(id = 'graph', figure = animated) #base graph
])
 
#callback function
@app.callback(
    Output('graph', 'figure'),
    Input('input1','value')
)
 
def update1(_input1):
    df_filtered = df.query('population_category == @_input1') #filtering for the chosen category
    animated = px.scatter(df_filtered, x="renewable_energy_consumption", y="co2_emission", animation_frame="Time",
                         size="population", hover_name="country")
 
    animated.update_layout(plot_bgcolor = 'white', 
                           xaxis_title='Renewable Energy Consumption (% of total energy comsumption)', #renaming the axis
                           yaxis_title='CO2 Emission (t CO2e/capita)',
                          title_subtitle_text = 'Filtered for population category, data source: World Bank Data: World Development Indicators')
    animated.update_layout(title_text=_input1)
    return animated

 
app.run(debug = True)

It seems like high populationed coutries move a lot along the x axis, but not so much on the y axis.
Medium populationed countries move along both axeses and there is no general tendency.
Very low populationed countries, they are way much more spread out than any other categories and they move on the axes without tendency.